In [36]:
import psycopg2
import pandas as pd
import pyspark
import os
import json
from pyspark.sql import SQLContext

In [7]:
try:
    conn = psycopg2.connect("dbname='gpadmin' user='gpadmin' host='gpdb' password='pivotal'")
    cur = conn.cursor()
    conn.autocommit = True
except:
    print "I am unable to connect to the database"

In [8]:
cur.execute("""SELECT madlib.version()""")
df = pd.DataFrame(cur.fetchall())
print df

                                                   0
0  MADlib version: 1.13, git revision: rel/v1.13,...


In [9]:
cur.execute("""DROP TABLE IF EXISTS public.testing""")
cur.execute("""CREATE TABLE public.testing (col1 float, col2 serial)""")
cur.execute("""INSERT INTO public.testing (SELECT random() AS col1 FROM (SELECT generate_series(1,100,1) AS col1) foo)""")

In [11]:
cur.execute("""SELECT count(*) FROM public.testing""")
cur.fetchall()[0]

(101L,)

In [62]:
query = """
    SELECT * FROM plcontainer_refresh_config;
"""
cur.execute(query)

In [63]:
query = """
    CREATE OR REPLACE FUNCTION pylog100() RETURNS double precision AS $$
    # container: plc_python_shared
    import math
    return math.log10(100)
    $$ LANGUAGE plcontainer;
"""
cur.execute(query)

In [64]:
query = """
    SELECT pylog100();
"""
cur.execute(query)
pd.DataFrame(cur.fetchall())

InternalError: Container 'plc_python_shared' is not defined in configuration and cannot be used (plcontainer.c:213)


In [4]:
# SparkSQL
sparkSQL = SQLContext(sc)

In [57]:
gpdf = spark.read.format("io.pivotal.greenplum.spark.GreenplumRelationProvider").options(
    url="jdbc:postgresql://gpdb:5432/gpadmin",
    user="gpadmin",
    password="pivotal",
    dbtable="testing",
    partitionColumn="col2").load()


In [58]:
# count number of records
gpdf.count()

101

In [ ]:
# persist dataframe in memory
gpdf.persist()

In [61]:
gpdf.toPandas()

AttributeError: 'DataFrame' object has no attribute 'show'